Notebook zur Ermittlung des Zusammenhangs aus Standardabweichung und Wachstumsrate.
Geforderte Eingabeparameter: 
    -Ordnerpfad mit beinhalteten Datensatzdateien
    -Name der Ausgabedatei
    -Anzahl der Windows
    -Threshold für Lücke
    -Threshold für Überschuss


In [2]:
from UltraClass import UltraClass
import re
import os
import xlsxwriter
folder = input("Ordner: ")
targetFile = input("Ausgabetabelle (inkl. .xlsx")
growthRatePattern = re.compile('(?<=_)(\d*?)\.(\d*?)(?=_)')
try:
    readsPerWindow = int(input("Reads pro Windows: "))
    thresholdLuecke = float(input("Threshold Lücke: "))
    thresholdUeberschuss = float(input("Threshold Überschuss: "))
except:
    print("Ungültige Eingabe")

print(f"Es wird der Ordner \"{folder}\" mit {readsPerWindow} Reads/Window analysiert. Thresholds für Lücke und Überschuss sind {thresholdLuecke} und {thresholdUeberschuss}")

Es wird der Ordner "Probedaten/Beispiesamples/Mail_lutz_3/Luecken/5_percent/" mit 80 Reads/Window analysiert. Thresholds für Lücke und Überschuss sind 1.0 und -1.0


In [3]:
results = {}

for filename in os.listdir(folder):
    if filename.endswith(".csv") and growthRatePattern.search(filename):
        fileAnalyse = UltraClass(folder+filename, thresholdLuecke, thresholdUeberschuss, readsPerWindow)
        datasetList, readAmountPerSection, readAmountPerSectionDict, readsPerSectionDict, xVectors, yVectors, xAxisDiagram = fileAnalyse.readFile()
        degreeDiffList, xList, avg, standardAbw, relEaList = fileAnalyse.calcWinkel(datasetList)
        gapBereiche = fileAnalyse.determineGaps(relEaList, datasetList)
        linReg1, linReg2, filledGaps, filledEllipse = fileAnalyse.fillGaps(gapBereiche, readAmountPerSection, xAxisDiagram)
        windowAbwDict, betterDataFileName = fileAnalyse.getWindowAbweichung(filledGaps[0], filledGaps[1], readAmountPerSectionDict, readsPerSectionDict)

        gapsFound = True
        timesReanalyzed = 0
        while gapsFound:
            timesReanalyzed += 1
            reAnalyse = UltraClass(betterDataFileName, thresholdLuecke, thresholdUeberschuss, readsPerWindow)
            reDatasetList = reAnalyse.readFile()
            reDegreeDiffList, reXList, reAvg, reStandardAbw, reRelEaList = reAnalyse.calcWinkel(reDatasetList[0])
            reGapBereiche = reAnalyse.determineGaps(reRelEaList, reDatasetList[0])
            reLinReg1, reLinReg2, reFilledGaps, reFilledEllipse = reAnalyse.fillGaps(reGapBereiche, reDatasetList[1], reDatasetList[6])
            reWindowAbw, betterDataFileName = reAnalyse.getWindowAbweichung(reFilledGaps[0], reFilledGaps[1], reDatasetList[2], reDatasetList[3], iteration=timesReanalyzed, originalFile=filename)
            
            if len(reFilledGaps[0]) == 0:
                gapsFound = False

        try:
            growthRate = float(growthRatePattern.search(filename).group())
        except:
            print("E: Regex-Match fehlgeschlagen")
        results[filename] = {"growth": growthRate, "standardAbw": reStandardAbw, "anzahlWindows" : reAnalyse.getAnzahlWindows()}
    else:
        print("E: Nicht alle Dateien sind CSV-Dateien oder besitzen den richtigen Dateinamen")


workbook = xlsxwriter.Workbook("Output/Growth/"+targetFile)
worksheet = workbook.add_worksheet("Data")

row = 0

for datasetName in results.keys():
    worksheet.write(row, 0, datasetName)
    worksheet.write(row, 1, results[datasetName]["growth"])
    worksheet.write(row, 2, results[datasetName]["standardAbw"])
    worksheet.write(row, 3, results[datasetName]["anzahlWindows"])
    worksheet.write(row, 4, readsPerWindow)
    row += 1
workbook.close()

KeyboardInterrupt: 